In [270]:
import cv2
from matplotlib import pyplot as plt
import numpy as np

In [271]:
def view_image(image):
    cv2.namedWindow('Display', 1)
    image_normal_size = cv2.resize(image, (960, 540))
    cv2.imshow('Display', image_normal_size)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [272]:
def HSL_color_selection(image):
    """
    Apply color selection to the HSL images to blackout everything except for white and yellow lane lines.
        Parameters:
            image: An np.array compatible with plt.imshow.
    """
    #Convert the input image to HSL
    converted_image = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
    
    #White color mask
    lower_threshold = np.uint8([0, 100, 0])
    upper_threshold = np.uint8([255, 255, 255])
    white_mask = cv2.inRange(converted_image, lower_threshold, upper_threshold)
    
    #Yellow color mask
    lower_threshold = np.uint8([10, 0, 100])
    upper_threshold = np.uint8([40, 255, 255])
    yellow_mask = cv2.inRange(converted_image, lower_threshold, upper_threshold)
    
    #Combine white and yellow masks
    mask = cv2.bitwise_or(white_mask, yellow_mask)
    masked_image = cv2.bitwise_and(image, image, mask = mask)
    
    return masked_image

In [273]:
def canny_detector(image, low_threshold = 50, high_threshold = 150):
    """
    Apply Canny Edge Detection algorithm to the input image.
        Parameters:
            image: An np.array compatible with plt.imshow.
            low_threshold (Default = 50).
            high_threshold (Default = 150).
    """
    return cv2.Canny(image, low_threshold, high_threshold)

In [274]:
def gray_scale(image):
    """
    Convert images to gray scale.
        Parameters:
            image: An np.array compatible with plt.imshow.
    """
    return cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

In [461]:
def region_selection(image):
    """
    Determine and cut the region of interest in the input image.
        Parameters:
            image: An np.array compatible with plt.imshow.
    """
    mask = np.zeros_like(image)   
    #Defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(image.shape) > 2:
        channel_count = image.shape[2]
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
    #We could have used fixed numbers as the vertices of the polygon,
    #but they will not be applicable to images with different dimesnions.
    rows, cols = image.shape[:2]
    bottom_left  = [cols * 0.01, rows * 0.8]
    top_left     = [cols * 0.01, rows * 0.7]
    bottom_right = [cols * 0.99, rows * 0.8]
    top_right    = [cols * 0.99, rows * 0.7]
    vertices = np.array([[bottom_left, top_left, top_right, bottom_right]], dtype=np.int32)
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    masked_image = cv2.bitwise_and(image, mask)
    return masked_image

In [456]:
def hough_transform(image):
    """
    Determine and cut the region of interest in the input image.
        Parameters:
            image: The output of a Canny transform.
    """
    rho = 1              #Distance resolution of the accumulator in pixels.
    theta = np.pi/180    #Angle resolution of the accumulator in radians.
    threshold = 20       #Only lines that are greater than threshold will be returned.
    minLineLength = 20   #Line segments shorter than that are rejected.
    maxLineGap = 300       #Maximum allowed gap between points on the same line to link them
    return cv2.HoughLinesP(image, rho = rho, theta = theta, threshold = threshold,minLineLength = minLineLength, maxLineGap = maxLineGap)

In [457]:
def draw_lines(image, lines, color = [255, 0, 0], thickness = 2):
    """
    Draw lines onto the input image.
        Parameters:
            image: An np.array compatible with plt.imshow.
            lines: The lines we want to draw.
            color (Default = red): Line color.
            thickness (Default = 2): Line thickness.
    """
    image = np.copy(image)
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(image, (x1, y1), (x2, y2), color, thickness)
    return image

In [438]:
def average_slope_intercept(lines, middle_x):
    """
    Find the slope and intercept of the left and right lanes of each image.
        Parameters:
            lines: The output lines from Hough Transform.
    """
    left_lines    = [] #(slope, intercept)
    left_weights  = [] #(length,)
    right_lines   = [] #(slope, intercept)
    right_weights = [] #(length,)
    
    for line in lines:
        for x1, y1, x2, y2 in line:
            if x1 == x2:
                continue
            slope = (y2 - y1) / (x2 - x1)
            if np.absolute(slope) < 0.1:
                continue
            print('Slope:', slope)
            intercept = y1 - (slope * x1)
            length = np.sqrt(((y2 - y1) ** 2) + ((x2 - x1) ** 2))
            if (slope < 0 and x1 < middle_x):
                left_lines.append((slope, intercept))
                left_weights.append((length))
            elif (slope > 0 and x1 > middle_x):
                right_lines.append((slope, intercept))
                right_weights.append((length))
            else:
                continue
    left_lane  = np.dot(left_weights,  left_lines) / np.sum(left_weights)  if len(left_weights) > 0 else None
    right_lane = np.dot(right_weights, right_lines) / np.sum(right_weights) if len(right_weights) > 0 else None
    return left_lane, right_lane

In [439]:
def pixel_points(y1, y2, line):
    """
    Converts the slope and intercept of each line into pixel points.
        Parameters:
            y1: y-value of the line's starting point.
            y2: y-value of the line's end point.
            line: The slope and intercept of the line.
    """
    if line is None:
        return None
    slope, intercept = line
    x1 = int((y1 - intercept)/slope)
    x2 = int((y2 - intercept)/slope)
    y1 = int(y1)
    y2 = int(y2)
    return ((x1, y1), (x2, y2))

In [440]:
def lane_lines(image, lines):
    """
    Create full lenght lines from pixel points.
        Parameters:
            image: The input test image.
            lines: The output lines from Hough Transform.
    """
    left_lane, right_lane = average_slope_intercept(lines, image.shape[0]/2)
    y1 = image.shape[0]
    y2 = y1 * 0.6
    left_line  = pixel_points(y1, y2, left_lane)
    right_line = pixel_points(y1, y2, right_lane)
    return left_line, right_line

def lane_lines2(image, lines):
    new_lines = []
    for line in lines:
        for x1, y1, x2, y2 in line:
            new_lines.append(((x1,y1),(x2, y2)))
    return new_lines
    
def draw_lane_lines(image, lines, color=[255, 0, 0], thickness=12):
    """
    Draw lines onto the input image.
        Parameters:
            image: The input test image.
            lines: The output lines from Hough Transform.
            color (Default = red): Line color.
            thickness (Default = 12): Line thickness. 
    """
    line_image = np.zeros_like(image)
    for line in lines:
        if line is not None:
            cv2.line(line_image, *line,  color, thickness)
    return cv2.addWeighted(image, 1.0, line_image, 1.0, 0.0)

In [441]:
def gaussian_smoothing(image, kernel_size = 13):
    """
    Apply Gaussian filter to the input image.
        Parameters:
            image: An np.array compatible with plt.imshow.
            kernel_size (Default = 13): The size of the Gaussian kernel will affect the performance of the detector.
            It must be an odd number (3, 5, 7, ...).
    """
    return cv2.GaussianBlur(image, (kernel_size, kernel_size), 0)

In [442]:
def process_frame(frame):
    color_selected_frame = HSL_color_selection(frame)
    gray_frame = gray_scale(color_selected_frame)
    gaussian_frame = gaussian_smoothing(color_selected_frame)
    edge_detected_frame = canny_detector(gaussian_frame)
    masked_frame = region_selection(edge_detected_frame)
    hough_lines = hough_transform(masked_frame)
    print('End of processing')
    proc_frame = draw_lane_lines(frame, lane_lines(frame, hough_lines))
    view_image(proc_frame)
    print('Line appended')
    print('End of appending')
    return proc_frame

In [469]:
frame = cv2.imread("./test_road3.png")

In [470]:
# color_selected_frame = HSL_color_selection(frame)
# edge_detected_frame = canny_detector(color_selected_frame, 250,1000)
# masked_frame = region_selection(edge_detected_frame)

In [ ]:
images = process_frame(frame)

End of processing
Slope: -0.2872340425531915
Slope: -0.30522088353413657
Slope: -0.30528846153846156
Slope: 0.6714975845410628
Slope: 0.6217228464419475
Slope: -0.3054892601431981
Slope: 0.6717171717171717
Slope: 0.6243654822335025
Slope: -0.2857142857142857
Slope: -0.2863961813842482
Slope: -0.3042505592841163
Slope: -1.234375
Slope: -0.36231884057971014
Slope: -0.28661087866108786
Slope: 0.2474747474747475
Slope: -0.29310344827586204
Slope: -1.3725490196078431
Slope: -0.3045356371490281
Slope: 0.6688311688311688
Slope: 0.22935779816513763
Slope: -1.0
Slope: 0.2328767123287671
Slope: -0.30454545454545456
Slope: -0.30606860158311344
Slope: -0.3056179775280899
Slope: 0.6030927835051546
Slope: 0.13877551020408163
Slope: -1.1142857142857143
Slope: 0.14606741573033707
Slope: 0.6716417910447762
Slope: -0.2932330827067669
Slope: 0.30718954248366015
Slope: -0.7837837837837838
Slope: 0.15948275862068967
Slope: -0.3250517598343685
Slope: 0.6745283018867925
Slope: -1.4901960784313726
Slope: 0.14